# Downloado dos dados do Calipso

O satélite Calipso é uma missão da NASA que foi lançada em 2006 com o objetivo de estudar as nuvens e as partículas suspensas na atmosfera da Terra. Uma das tecnologias que ele usa para isso é o LIDAR (Light Detection and Ranging), que é capaz de medir a altura e a densidade dessas partículas com alta precisão.

## Objetivo:
O objetivo desse código é automatizar o download dos protudos do calipso de forma rápida e fácil.

## Requisitos:
Para que o código funcione adequadamente, você precisa já ter feito cadastro do site da Nasa e possuir os dados.


### Informações para contato:
#### Autor: André Luiz da Silva Amorim
##### Email: amorimandre2012@gmail.com

In [7]:
# Importações para automação web:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Importação pra trabalhar com o sistema
import os
import sys

# Importação para barra de progresso:
from tqdm import tqdm

# Importação administrar os downloads:
import time

# Class para download dos dados:

In [8]:
class CalipsoDownloader:
    '''
        A classe Downloader foi construida para lidar com os donwloads
    dos arquivos HDF para os dois casos que utilizamos, onde o donwload dos
    dados de Level 2 é estruturado de uma forma diferente do download dos 
    dados de Level 3.
    '''
    def __init__(self, folder_path, page, email=None, senha=None):
        self.folder_path = folder_path
        self.page = page
        self.email = email
        self.senha = senha
        self.opcoes = Options()
        self.opcoes.add_experimental_option("prefs", {
          "download.default_directory": self.folder_path,
          "download.prompt_for_download": False,
          "download.directory_upgrade": True,
          "safebrowsing.enabled": True
        })
        self.servico = Service(ChromeDriverManager().install())
        self.navegador = webdriver.Chrome(options=self.opcoes,service=self.servico)
    
    def downloadL2(self):
        try:
            self.navegador.get(self.page)
        except:
            print('Ih rapaz... Inseriu o url corretamente?')
        
        elements = self.navegador.find_elements('xpath',"/html/body/table/tbody/tr[*]/td[2]/a")
        hrefs = [element.get_attribute("href") for element in elements]
        dtime = 5
        for href in tqdm(hrefs):
            if href.endswith(".hdf"):
                done = False
                while not done:
                    try:
                        self.navegador.get(href)
                        time.sleep(dtime)
                        done=True
                    except WebDriverException:
                        time.sleep(dtime)
                        print(f"Erro ao acessar o link, tentando novamente em {dtime} segundos...")
                        self.navegador.get(href)
                        attempt += 1
                        print(f'{attempt} tentativa do arquivo {arquivo}')
                        
        # Checando se os donwloads terminaram:
        self.wait_download(self.folder_path)
        
        # Fechando o navegador:
        self.navegador.quit()
        print('Tudo Baixado')
        
        
    def downloadL3(self):
        self.navegador.get(self.page)
        self.navegador.find_element('xpath', '//*[@id="username"]')\
            .send_keys(self.email)
        self.navegador.find_element('xpath', '//*[@id="password"]')\
            .send_keys(self.senha)
        self.navegador.find_element('xpath', '//*[@id="login"]/p[8]/input')\
            .click()
        time.sleep(25)
        expand_botton = self.navegador.find_element('xpath',
            '//*[@id="app"]/div/div/div/div/section/div/div/div[2]/ul/li/div/div[2]/div/div[1]/div[2]/header/button[3]/span')
        expand_botton.click()
        elements = self.navegador.find_elements('xpath', '/html/body/div[7]/div/div/div[2]/div/div/ul/li[*]/a')
        hrefs = [element.get_attribute("href") for element in elements]
        dtime = 5
        for href in tqdm(hrefs):
            done=False
            attempt = 0
            arquivo = 0
            while not done:
                try:
                    self.navegador.get(href)
                    time.sleep(dtime)
                    done = True
                except WebDriverException:
                    print(f"Erro ao acessar o link, tentando novamente em {dtime} segundos...")
                    time.sleep(dtime)
                    self.navegador.get(href)
                    attempt += 1
                    arquivo += 1
                    print(f'{attempt} tentativa do arquivo {arquivo}')
        # Checando se os donwloads terminaram:
        self.wait_download(self.folder_path)
        
        # Fechando o navegador:
        self.navegador.quit()
        print('Tudo Baixado')
        
    def wait_download(self, download_dir):
        '''
            Método somente para checar se os arquivos foram todos
        baixados antes de terminar.
        '''
        
        while any([filename.endswith(".crdownload") for filename in os.listdir(download_dir)]):
            time.sleep(1)

# Download:

In [9]:
# Input para definição do tipo de download: 
print('''
    Vale ressaltar que a forma de se baixar os dados do L2 servem
para o L1 também, ou seja, L2 como resposta também serve como resposta para L1
''')
escolha = input('Qual tipo de dado gostaria de baixar? (L2/L3)')

# Checagem da entrada:
if escolha != "L2" and escolha != "L3":
    print('Essa é uma escolha binária, ou é L2 ou L3')
    sys.exit()
    
#inputs:
diretorio = input('Digite o diretório para os arquivos\n')
pasta = input('Digite a pasta onde ficara esse arquivo') #Recomendação: Digite um nome significativo.
pagina = input('Insira a url da página que se queria acessar\n')

if diretorio == '':
    diretorio = 'D:\Estudo\LASA\Dados\Bahia'
    
if pasta == '':
    pasta = 'Dados_L3'
    

# Configurações da Pasta
folder_path = os.path.join(diretorio, pasta)
if not os.path.exists(folder_path):
    os.makedirs(folder_path)


# Instanciando a classe correspondente à escolha do usuário
if escolha == "L2":
    downloader = CalipsoDownloader(folder_path, pagina).downloadL2()
    
if escolha == "L3":
    email = input('Insira seu email:\n')
    
    if email == "":
        try:
            email = email
        except:
            print('Informe o email')
        
    senha = input('Insira sua senha:\n')
    
    if senha == '':
        try:
            senha = senha
        except:
            print('Informe a senha')
    downloader = CalipsoDownloader(folder_path, pagina, email, senha).downloadL3()


    Vale ressaltar que a forma de se baixar os dados do L2 servem
para o L1 também, ou seja, L2 como resposta também serve como resposta para L1

Qual tipo de dado gostaria de baixar? (L2/L3)L3
Digite o diretório para os arquivos
D:\Estudo\LASA\Dados\Bahia
Digite a pasta onde ficara esse arquivoL3Night
Insira a url da página que se queria acessar
https://search.earthdata.nasa.gov/downloads/4320064146
Insira seu email:

Insira sua senha:



100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [01:40<00:00,  5.94s/it]


Tudo Baixado
